# Wyjaśnienia lokalne
Grupa BarteKasiAdam

## Wczytanie pakietów, danych i modelu

In [ ]:
import dalex as dx
import pandas as pd
import pickle
import xgboost as xgb
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
input_df = pd.read_csv('data/Bartek/new_preprocessed_dataset.csv')
y = input_df.loc[:,'Attrition']
X = input_df.drop('Attrition', axis='columns')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14)

In [ ]:
path_xgb = 'modele/new_xgb_model.p'
xgb = pickle.load(open( path_xgb, "rb" ))
xgb_explainer = dx.Explainer(xgb, X_train, y_train, label='XGB')

In [ ]:
path_rf = 'modele/new_random_forest_model.p'
rf = pickle.load(open( path_rf, "rb" ))
rf_explainer = dx.Explainer(rf, X_train, y_train, label='RandomForest')

## Obserwacja błędnie zaklasyfikowana przez oba modele

In [ ]:
xgb_errors = np.round(xgb_explainer.predict(X_test)) != y_test
rf_errors = np.round(rf_explainer.predict(X_test)) != y_test
incorrect = X_test[xgb_errors & rf_errors]

In [ ]:
incorrect.index

Powyżej wypisane są indeksy obserwacji ze zbioru testowego, które są błędnie zaklasyfikowane przez oba modele.

In [ ]:
index = 1257

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('XGB prediction : ' + str(xgb_explainer.predict(observation)[0]))
print('RF prediction : ' + str(rf_explainer.predict(observation)[0]))

Widzimy, że klient wypowiedział usługi, ale zarówno XGBoost jak i Random Forest pomyliły się. Przewidziały prawdopodobieństwo odejścia odpowiednio równe 0.5% oraz 4%.

### Break Down

In [ ]:
xgb_pp = xgb_explainer.predict_parts(X_test.loc[index])
rf_pp = rf_explainer.predict_parts(X_test.loc[index])

xgb_pp.plot(max_vars=24)
rf_pp.plot(max_vars=24)

XGBost wykrył, że `Total_Realtionship_Count` = 5 (Łączna liczba produktów w banku) pod warunkiem, że `Total_Trans_Amt`= 1096 (Łączna wartość transakcji) obniża prawdopodobieństwo odejścia klienta o około 3%. Random Forest natomiast nie wykrył interakcji miedzy tymi zmiennymi i uznał, że `Total_Trans_Amt`= 1096 o 7% zwiększa prawdopodobieństwo odejścia z banku. Z kolei `Total_Relationship_Count` = 5 obniżył predykcję o około 6%. Największy wpływ na odpowiedź XGBoost'a według metody Break Down miała wartość w kolumnie `Months_Inactive_12_mon` (liczba miesięcy bez aktywności w ciągu roku) równa 1 (zmniejszenie wyniku o 8%).

### Shapley Values

In [ ]:
xgb_shap = xgb_explainer.predict_parts(X_test.loc[index], type='shap')
rf_shap = rf_explainer.predict_parts(X_test.loc[index], type='shap')

xgb_shap.plot(max_vars=24)
rf_shap.plot(max_vars=24)

Według metody Shapley Values 5 zmiennych mających największy wpływ na predykcję obu modeli było takie same. Co więcej, znak kontrybucji każdej z tych zmiennych był taki sam dla dwóch modeli. Dokładniej, tylko `Total_Trans_Amt` zwiększał prawdopodobieństwo odejśca. `Total_Revolving_Bal`, `Months_Inactive_12_mon`, `Total_Relationship_Count` oraz `Contacts_Count_12_mon`(liczba kontaktów z bankiem w ciągu ostatniego roku) zmniejszały predykcję.

Modele dobrze zinterpretowały wartość w `Total_Trans_Amt`, ale wartości innych zmiennych zaważyły na tym, że predykcja była błędna.

### LIME

In [ ]:
xgb_lime = xgb_explainer.predict_surrogate(X_test.loc[index], type='lime', seed=10)
rf_lime = rf_explainer.predict_surrogate(X_test.loc[index], type='lime', seed=10)
xgb_lime.show_in_notebook()
rf_lime.show_in_notebook()

#### XGBoost
Metoda LIME pokazuje, że ważne w predykcji były `Total_Relationship_Count`, `Contacts_Count_12_mon`, `Total_Ct_Chng_Q4_Q1`, `Maritial_Divorced`. O ile pierwsze dwie nie są zaskoczeniem, gdyż wykrywały je już poprzednie metody, to kolejne mogą być dla nas zaskoczeniem, szczególnie `Maritial_Divorced` - flaga utworzona przy one hot encodingu. W tej analizie jej wartość równa 0 zwiększa predykcję. Innymi słowy, to że klient nie jest rozwiedziony zwiększa prawdopodobieństwo odejścia z banku.

#### Random Forest
Tutaj nie ma zaskoczenia: 5 najważniejszych zmiennych wykrytych metodą LIME jest taka sama jak w przypadku metody Shapley Values. Również kierunek ich kontrybucji jest identyczny jak w poprzedniej metodzie.

### Ceteris Paribus

In [ ]:
xgb_cp = xgb_explainer.predict_profile(X_test.loc[index])
rf_cp = rf_explainer.predict_profile(X_test.loc[index])

xgb_cp.plot(rf_cp, facet_ncol=4)

Wnioski po przeanalizowaniu profili Ceteris Paribus:
- Random Forest jest bardziej ważliwy na odchylenia wartości zmiennych. Przy odchyleniu wartości zmiennych zmiana jego predykcji jest większa niż w przypadku XGBoost'a.
- aby wynik modeli zgadzał się z etykietą, musiałoby zajść jedno ze zdarzeń: 
    + `Contact_Count_12_mon` = 6 
    + `Total_Trans_Amt` < 850
    
    A więc musiałaby nastąpić zmiana wartości kolumn, które zostały wskazane za istotne w analizach innymi metodami. Możemy stąd wnioskować, że rzeczywiście te kolumny miały wpływ na wynik modeli
    
### Podsumowanie

- kolumna `Total_Trans_Amt` wpływa na wynik w dobrym kierunku według prawie wszystkich analiz, ale jej kontrybucja jest zagłuszana przez inne zmienne.
- najważniejszymi kolumnami wpływającymi na wynik dla tej obserwacji są `Total_Trans_Amt`, `Total_Revolving_Bal`, `Months_Inactive_12_mon`, `Total_Relationship_Count` oraz `Contacts_Count_12_mon`.
- błędna predykcja modeli wynikaja w większości z tych samych powodów.

## Obserwacja bardzo dobrze klasyfikowana

In [ ]:
treshhold = 0.00005

xgb_errors = np.abs(xgb_explainer.predict(X_test) - y_test)
rf_errors = np.abs(rf_explainer.predict(X_test) - y_test)
super_correct = X_test[(xgb_errors < treshhold) & (rf_errors < treshhold)]

In [ ]:
super_correct.index

Powyżej wypisane są indeksy obserwacji ze zbioru testowego, które są bardzo dobrze zaklasyfikowane przez oba modele.

In [ ]:
index = 8024

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('XGB prediction : ' + str(xgb_explainer.predict(observation)[0]))
print('RF prediction : ' + str(rf_explainer.predict(observation)[0]))

Widzimy, że klient dalej korzysta z usługi, a zarówno XGBoost jak i Random Forest były całkowicie przekonane do tego, że właśnie tak będzie.
Sprawdźmy czemu są aż tak tego pewni.

### Break Down

In [ ]:
xgb_pp = xgb_explainer.predict_parts(X_test.loc[index])
rf_pp = rf_explainer.predict_parts(X_test.loc[index])

xgb_pp.plot(max_vars=11)
rf_pp.plot(max_vars=11)

Modele wykryły, że `Total_Realtionship_Count` = 1 (Łączna liczba produktów w banku). Uznały, że to niewiele, więc być może klient ten nie jest związany z naszym bankiem i rozważa opuszczenie go. Random Forest uznał go za najbardziej wpływową kolumnę w pierwszym kroku. XGBost natomiast w drugim, ale nadal niesamowicie bardzo.

A jednak ostatecznie modele uznały, że predykcja będzie przeciwna do tego, co sugeruje ta kolumna. Czemu tak się stało? bo `Total_Trans_Amt` = 4410 (łączna wartość przelewów). To była kolumna, która dla Random Forest całkowicie zniwelowała "przeczucia" dla tamtej kolumny, a dla XGBost było to 4 krotnie ważniejsze.

Czemu jednak akurat taka wartość kolumny `Total_Trans_Amt` przekonuje model co do pozostania klienta w banku?

### Ceteris Paribus

In [ ]:
xgb_cp = xgb_explainer.predict_profile(X_test.loc[index])
rf_cp = rf_explainer.predict_profile(X_test.loc[index])

xgb_cp.plot(rf_cp, facet_ncol=4)

Okazuje się, że według modelu XGBoost, gdyby wszystko zostało w tej obserwacji takie samo, tylko wartość `Total_Trans_Amt` zmienić na 0, to model byłby prawie $100\%$ pewien, że klient odejdzie z naszego banku, a tak, dzięki temu, że jest na poziomie $4400$, jest prawie $100\%$ pewnie, że nie opuści. Dla Random Forest to też jest bardzo ważne, co prawda nie aż do $100\%$, ale $60\%$, co i tak jest ogromną wartością.

Oznacza to, że klienci, którzy bardzo niewiele kożystają z podstawowej usługi bankowej, jaką jest wykonywanie przelewów, są kategoryzowani jako opuszczający bank.

Widzimy dodatkowo, że gdyby zwiększyć wartość `Contacts_Count_12_mon`, czyli liczbę odwiedzin siedziby banku, to równierz diametralnie zmieniają się przewidywania odnośnie odejścia. Zgadza się to z intuicją, że opuszczenie banku przez klienta to długoterminowy proces, który wymaka kilku odwiedzin w placówce, choćby, żeby wypłacić pozostałą gotówkę. Klient, który wielokrotnie odwiedza placówkę to taki, który próbuje coś ważnego załatwić i być może jest to opuszczenie banku. Być może jednak klient często odwiedza fizyczną placówkę, bo aplikcaja mobilna banku nie pozwala mu wykonać jakiejś tranzakcji, albo nie jest wystarczająco intuicyjna, aby mu to umożliwić. Dlatego swoje sprawy załatwia osobiście. Taki klient może się zniechęcić do korzystania z usług bankowych i poszukać na runku alternatywy.

### Shapley Values

In [ ]:
xgb_shap = xgb_explainer.predict_parts(X_test.loc[index], type='shap')
rf_shap = rf_explainer.predict_parts(X_test.loc[index], type='shap')

xgb_shap.plot(max_vars=24)
rf_shap.plot(max_vars=24)

Według metody Shapley Values 4 zmienne mających największy wpływ na predykcję obu modeli było takie same. Co więcej, znak kontrybucji każdej z tych zmiennych był taki sam dla dwóch modeli.

Oznacza to, że modele najbardziej biorą pod uwagę te same kolumny, jednak z inną "mocą".

Co ciekawe modele uwzględniły, że wartość `Total_Revolving_Bal` na poziomie około $1600$ powoduje "spadek" predykcji o około $0.03$ dla XGBoost i $0.06$ dla Random Forest. Wydaje się, że jeśli klient zostawił na kolejny okres rozliczeniowy aż $1600$ zł długu, to będzie dużo bardziej rokował na długotrwałe przywiązanie się do instytucji bankowej, niż tylko $0.03$. Tym bardziej, że według wykresu Ceteris Paribus dla Random Forest niska jego wartość powoduje "skok" predykcji aż do $0.4$.

### LIME

In [ ]:
xgb_lime = xgb_explainer.predict_surrogate(X_test.loc[index], type='lime', seed=10)
rf_lime = rf_explainer.predict_surrogate(X_test.loc[index], type='lime', seed=10)
xgb_lime.show_in_notebook()
rf_lime.show_in_notebook()

Metoda LIME potwierdza ważność wcześniej analizowanych kolumn. Co jednak ciekawe, wartość kolumny `Total_Revolving_Bal` wegług modelu XGBoost jest tym razem oceniona jako druga pod względem istotności.

### Podsumowanie
- kolumna `Total_Realtionship_Count` = 1 powoduje, że bank boi się, że nie trzyma klienta wystarczająco mocno i że klient rozważa opuszczenie go. Jednakże okazuje się, iż jest to klient, który chętnie trzyma w tym banku swoje pieniądze i nie myśli o opuszczaniu go mimo.
- najważniejszymi kolumnami wpływającymi na wynik dla tej obserwacji są `Total_Trans_Amt`, `Total_Relationship_Count`, `Total_Revolving_Bal`.
- pewność predykcji jest spowodowana dużym obrotem na koncie klienta oraz sporymi pożyczkami nierozliczanymi w terminie.

## Obserwacja niepewnie zaklasyfikowana przez oba modele

In [ ]:
xgb_ff_a = xgb_explainer.predict(X_test) > 0.42 
rf_ff_a = rf_explainer.predict(X_test) > 0.42
xgb_ff_b = xgb_explainer.predict(X_test) < 0.57
rf_ff_b = rf_explainer.predict(X_test) < 0.57
ff = X_test[xgb_ff_a & rf_ff_a & xgb_ff_b & rf_ff_b]


In [ ]:
ff.index

In [ ]:
index = 5673

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('XGB prediction : ' + str(xgb_explainer.predict(observation)[0]))
print('RF prediction : ' + str(rf_explainer.predict(observation)[0]))

In [ ]:
index = 6249

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('XGB prediction : ' + str(xgb_explainer.predict(observation)[0]))
print('RF prediction : ' + str(rf_explainer.predict(observation)[0]))

In [ ]:
index = 3363

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('XGB prediction : ' + str(xgb_explainer.predict(observation)[0]))
print('RF prediction : ' + str(rf_explainer.predict(observation)[0]))

In [ ]:
index = 8762

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('XGB prediction : ' + str(xgb_explainer.predict(observation)[0]))
print('RF prediction : ' + str(rf_explainer.predict(observation)[0]))

Sprawdzając obserwacje których predykcja dla obu modeli jest bliks 50%, zaóważyłam że dotyczą one często klientów którzy zrezygnowali.

In [ ]:
index = 4982

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('XGB prediction : ' + str(xgb_explainer.predict(observation)[0]))
print('RF prediction : ' + str(rf_explainer.predict(observation)[0]))

Klient zrezygnował z usługi. Oba modele skłąnią się delikatnie w stronę rezygnacji klienta, lecz jest to bardzo nie pewna decyzja.

### Break Down

In [ ]:
xgb_pp = xgb_explainer.predict_parts(X_test.loc[index])
rf_pp = rf_explainer.predict_parts(X_test.loc[index])

xgb_pp.plot(max_vars=24)
rf_pp.plot(max_vars=24)

Dekompozycję zaczynamy od wartosci 0.159, wieć na początku nasza predykcja uważa, że klient nie chcę zrezygnować. Przyjrzyjmy się jak nasza obserwacja próbuje przekonać modele, że jednak ma zamair zrezygnować z usługo.
Pierwsze i najważniejsze na co nasze modele zwracają uwagę to `Total_Revolving_Bal` = 0 oraz `Total_Trans_Amt` = 2709. 
Brak srodków na koncie i niska suma transakcji z ostatnich 12 miesięcy może być dobrą wskazówka by stwierdzić, że klijent chce zrezygnować.Dwie zmienne mówią jednak co innego. `Avg_Open_To_Buy` to różnica między limitem karty a stanem konta. Dla naszej obserwacji `Avg_Open_To_Buy` = 25670. Drugą zmienną jest `Avg_Utilization_Rate`. Jest to stosunek nie spłaconych kwot do limitu karty i dla naszej obserwacji jest równe 0. Oznacza to brak wykorzystanego debetu.


### Shapley Values

In [ ]:
xgb_shap = xgb_explainer.predict_parts(X_test.loc[index], type='shap')
rf_shap = rf_explainer.predict_parts(X_test.loc[index], type='shap')

xgb_shap.plot(max_vars=24)
rf_shap.plot(max_vars=24)

Dla Shapley Value ważne zmienne są podobne co dla Break Down. Dodatkowo widać udział `Months_Inactive_12_mon` oraz `Total_Amt_Chng_Q4_Q1`.3 miesiące nie aktywności może zrozumieć jako brak zainteresowania usługą, więc ma sens przełożenie miało to dodatni wpływ na predykcje rezygnacji. `Total_Amt_Chng_Q4_Q1` oznacza ile razy wzrosła kwota transakcji w przeciągu 12 miesięcy. Dla naszej obserwacji wartość ta jest równa prawię 0.95. Może to oznaczać, że klient aktywnie używa swojego konta.

Zaóważmy jeszcze, że dla Xgboosta `Total_Trans_Amt` miało większy wpływ niż `Total_Revolving_Bal`, a dla RandomForest podobne.

### LIME

In [ ]:
xgb_lime = xgb_explainer.predict_surrogate(X_test.loc[index], type='lime', seed=10)
rf_lime = rf_explainer.predict_surrogate(X_test.loc[index], type='lime', seed=10)
xgb_lime.show_in_notebook()
rf_lime.show_in_notebook()

W dekompozycji LIME , oprócz innych znanych nam już zmiennych zaóważalny wpływ ma `Total_Relatioship_Count` = 3.
Patrząc na liczę zmiennych, które przemawiają za predykcją o rezygnacji oraz ich wpływ na predykcję można by pomyśleć, że modele przewidziały rezygnację z usługi.


### Ceteris Paribus


In [ ]:
xgb_cp = xgb_explainer.predict_profile(X_test.loc[index])
rf_cp = rf_explainer.predict_profile(X_test.loc[index])

xgb_cp.plot(rf_cp, facet_ncol=4)

Nasza obserwacja dla wielu zmiennych przyjmuje wartosci które znajdują się pomiędzy skokami. Dla modelu xgboost wystarczą niewielkie zmiany by predykcja była bliska 1.
Gdyby dla naszej obserwacji `Total_Ct_Chng_Q4_Q1` miało by mniejszą wartość(spadek liczby transakcji z czasem, co może oznaczać spadek zainteresowania), to model xgboost z wysoką predykcją przewidział by rezygnacje klienta.  Do lepszej predykcji przyczyniło by się także zmiejszenie `Avg_open_To_Buy`, które jest dosyć wysokie jak na osobę która rezygnuje z usługi. Gdyby `Contacts_Count_12_mon` było większe, czyli klient cześciej predykcja rezygnacji bedzie większa. Wystarczy więc by jedna z tych zmienuch miała trochę inną wartość by osiągnąć dobrą predykcję. Niestety dla zmiennych ktorych zmiany wartości powodują duże zminy predykcji, nasza obserwacja przyjmuje "wartości średnie" który zwykle przekładają się na predykcję bliską 50%.

### Podsumowanie
- dużo obserwacje o najwiekszej niepewności predykcji dotyczą klijentów którzy zrezygnowali
- najwiekszy wpływ ma `Total_Revolving_Bal` oraz `Total_Trans_Amt` 
- niepewna predykcj może wynikać z braku innych zmiennych o wpływowych wartościach 